In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 132kB/s 
     |████████████████████████████████| 6.8MB 52.0MB/s 
     |████████████████████████████████| 983kB 48.4MB/s 
     |████████████████████████████████| 266kB 52.0MB/s 
     |████████████████████████████████| 1.3MB 45.6MB/s 
     |████████████████████████████████| 1.2MB 45.8MB/s 
     |████████████████████████████████| 471kB 48.0MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
     |████████████████████████████████| 2.9MB 45.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=001df2d4cbca82b0f942f80fe005bff00e8da56774daa2d78541c56e15dc23ee
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=d7fc2fc150d6e521e4be4dd97560f2914503f7954a28b4da5748129bd7880726
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [6]:
data_train = pd.read_excel('https://github.com/jenhuluck/NLP_projects/blob/main/IMDB%20movie%20review%20sentiment%20classification/train.xlsx?raw=true', dtype= str)
data_test = pd.read_excel('https://github.com/jenhuluck/NLP_projects/blob/main/IMDB%20movie%20review%20sentiment%20classification/test.xlsx?raw=true', dtype = str)


In [7]:
data_train.sample(5)

,Reviews,Sentiment
7273,"I have seen poor movies in my time, but this r...",neg
18691,"I always found Betsy Drake rather creepy, and ...",neg
17899,I don't remember ever seeing this one before t...,pos
19788,This movie is great entertainment to watch wit...,pos
8270,This was truly dreadful! It had a terrible sto...,neg


In [8]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [9]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment',
                   val_df = data_test,
                   maxlen = 400,
                   preprocess_mode = 'distilbert')


preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [10]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [11]:
learner = ktrain.get_learner(model = model,
                             train_data = train,
                             val_data = val,
                             batch_size = 6)

In [12]:
learner.fit_onecycle(lr = 2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4167/4167 [==============================] - 1008s 239ms/step - loss: 0.3767 - accuracy: 0.8253 - val_loss: 0.1549 - val_accuracy: 0.9488
Epoch 2/2
4167/4167 [==============================] - 999s 239ms/step - loss: 0.1705 - accuracy: 0.9365 - val_loss: 0.0654 - val_accuracy: 0.9821


In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
predictor.save('/content/drive/My Drive/distilbert')

In [16]:
data = ['this movie was really bad. acting was also bad. I will not watch again',
        'the movie was really great. I will see it again', 'another great movie. must watch to everyone']

In [17]:
predictor.predict(data)

['neg', 'pos', 'pos']

In [19]:
predictor.get_classes()

['neg', 'pos']

In [20]:
predictor.predict(data, return_proba=True)

array([[0.99587643, 0.00412357],
       [0.00740423, 0.99259585],
       [0.00388876, 0.9961112 ]], dtype=float32)